<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Методы машинного обучения</center></h1>
<h2><center>Тематическое моделирование</center></h2>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

# Работа с текстом

In [ ]:
df = pd.read_csv('./data/labeledTrainData.tsv', sep='\t')
df.head()

### Стандартная предобработка из предыдущего занятия

In [ ]:
## Оставляем только слова из латиницы
import re
regex = re.compile(u"[A-Wa-z]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


df.review = df.review.str.lower()
df.loc[:, 'review'] = df.review.apply(words_only)

In [ ]:
import nltk

In [ ]:
## Удаляем стоп-слова
from nltk.corpus import stopwords
mystopwords = stopwords.words('english') + ['br']

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return u" ".join([token for token in nltk.word_tokenize(text) if not token in mystopwords])
    except:
        return u""
    
df.review = df.review.apply(remove_stopwords)   

In [ ]:
df.review.sample(10)

In [ ]:
## Нормализуем текст
wnl = nltk.WordNetLemmatizer()
print(wnl.lemmatize('loved'))

In [ ]:
## Нормализуем текст
stemmer = nltk.PorterStemmer()
print(stemmer.stem('loved'))

In [ ]:
%%time 

def normalize(text, method=stemmer):
    try:
        return " ".join(method.stem(word) for word in text.split(' '))
    except:
        return " "
    
df.loc[:, 'review_normed'] = df.review.apply(normalize)

In [ ]:
df.loc[:, 'review_normed'].sample(10)

## Тематическое моделирование
На данный момент, наиболее популярные библиотеки для тематического моделирования, это:
* [Gensim](https://radimrehurek.com/gensim/)
* [BigARTM](https://bigartm.readthedocs.io/en/stable/index.html)

Сегодня (а может и никогда) BigARTM мы разбирать не будем, по той простой причине, что в [репозитории с примерами](https://github.com/bigartm/bigartm-book/) довольно много материалов, в которых содержится исчерпывающая информация о том, как что работает (например [тут](https://github.com/bigartm/bigartm-book/blob/master/ARTM_tutorial_Fun.ipynb) или [тут](https://github.com/bigartm/bigartm-book/blob/master/ARTM_example_RU.ipynb) )

In [ ]:
from gensim.corpora import *
texts = [df.review_normed.iloc[i].split() for i in range(len(df))]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
from gensim.models import ldamodel

In [ ]:
%%time
lda = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, 
                        num_topics=20,
                        alpha='auto', eta='auto', 
                        iterations = 20, passes = 10)

In [ ]:
## Достаем описание тем

topics = lda.show_topics(20)
for t in range(20):
    print('==========')
    print(topics[t][1])

In [ ]:
## Достаем описание документов
from gensim import matutils
T = matutils.corpus2dense(lda[corpus], 20).T

##  Визуализация тем
[pyLDAvis](https://pyldavis.readthedocs.io/en/latest/readme.html)

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)